In [1]:
import pandas as pd
import joblib 
import numpy as np 
import datetime as datetime
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import joblib
from pathlib import Path


In [2]:

file_path = Path("/Users/lorelablaka/Documents/Flatiron/phase-5/capstone/collect_clean")

age = joblib.load(f"{file_path}/age.pkl")
sec_10k = joblib.load(f"{file_path}/sec_10k.pkl")
managers = joblib.load(f"{file_path}/managers_all.pkl")
yf1 = joblib.load(f"{file_path}/yf1.pkl")
yf2 = joblib.load(f"{file_path}/yf2.pkl")

**EDA of Datasets**

In [3]:
dfs = [age,sec_10k,managers,yf1,yf2]
df_names = ['age','sec_10k','managers','yf1','yf2']

for df in dfs:
    unique_list = list(df['ticker'].unique())
    print(f"{len(set(unique_list))}  unique tickers and {len(df)} rows in dataset")
    
    
 


10686  unique tickers and 10686 rows in dataset
2528  unique tickers and 114475 rows in dataset
3543  unique tickers and 3543 rows in dataset
8682  unique tickers and 8682 rows in dataset
3216  unique tickers and 3420 rows in dataset


**Concatinate `yahoofinance` IPO data with `managers` IPO data**

In [4]:
df_man = managers[['ticker','trade_date','open','close']]
df_man.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3543 entries, 0 to 3542
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ticker      3543 non-null   object        
 1   trade_date  3542 non-null   datetime64[ns]
 2   open        3543 non-null   float64       
 3   close       3543 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 138.4+ KB


In [5]:
def ipo_returns(df,col1,col2):
    ret = (df[col2] - df[col1] )/df[col1]
    return ret

In [6]:
df_man['day_1_%'] = ipo_returns(df_man,'open','close')

<ipython-input-6-60729fe8924f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_man['day_1_%'] = ipo_returns(df_man,'open','close')


In [7]:
df_man =df_man[['ticker','trade_date','day_1_%']]

In [8]:
all_ipos = (yf2,df_man)
all_ipos = pd.concat(all_ipos)
all_ipos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6963 entries, 0 to 3542
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ticker      6963 non-null   object        
 1   trade_date  6962 non-null   datetime64[ns]
 2   day_1_%     6963 non-null   float64       
 3   month_7_%   2588 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 272.0+ KB


Significantly less rows for `month_7_%`  

In [9]:
all_ipos= all_ipos.groupby(['ticker']).max().reset_index()
all_ipos

,ticker,trade_date,day_1_%,month_7_%
0,SERV,2014-06-26,0.028653,NaN
1,A,1999-11-18,-0.032967,-0.009236
2,AA,1962-01-02,inf,0.051095
3,AAACU,2006-04-11,0.080460,NaN
4,AAAP,2015-11-11,0.324324,NaN
...,...,...,...,...
6430,ZX,2011-05-12,-0.025000,NaN
6431,ZYME,2017-04-28,-0.037037,NaN
6432,ZYNE,2015-08-05,-0.097222,NaN
6433,ZZ,2006-04-07,0.040000,0.078161


**Merging `all_ipo` data with recent company financials from `yf1` dataset**

In [10]:
yf =  all_ipos.merge(yf1, how='left',on='ticker')
yf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6435 entries, 0 to 6434
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ticker                 6435 non-null   object        
 1   trade_date             6434 non-null   datetime64[ns]
 2   day_1_%                6435 non-null   float64       
 3   month_7_%              2392 non-null   float64       
 4   total_revenue          5482 non-null   float64       
 5   operating_cashflow     5482 non-null   float64       
 6   full_time_employees    5482 non-null   float64       
 7   sector                 5451 non-null   object        
 8   revenue_growth         4937 non-null   float64       
 9   gross_margins          5442 non-null   float64       
 10  profit_margins         5263 non-null   float64       
 11  enterprise_to_revenue  4937 non-null   float64       
 12  ebitda_margins         5442 non-null   float64       
 13  mar

In [11]:
yf['ipo_year']= yf['trade_date'].dt.year
yf['ipo_year']

0       2014.0
1       1999.0
2       1962.0
3       2006.0
4       2015.0
         ...  
6430    2011.0
6431    2017.0
6432    2015.0
6433    2006.0
6434    2009.0
Name: ipo_year, Length: 6435, dtype: float64

**Merging `all_ipo` data and recent company financials with `age` dataset to include stock features**

In [12]:
data2 = yf.merge(age, how='left',on=['ticker'])
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6435 entries, 0 to 6434
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ticker                 6435 non-null   object        
 1   trade_date             6434 non-null   datetime64[ns]
 2   day_1_%                6435 non-null   float64       
 3   month_7_%              2392 non-null   float64       
 4   total_revenue          5482 non-null   float64       
 5   operating_cashflow     5482 non-null   float64       
 6   full_time_employees    5482 non-null   float64       
 7   sector                 5451 non-null   object        
 8   revenue_growth         4937 non-null   float64       
 9   gross_margins          5442 non-null   float64       
 10  profit_margins         5263 non-null   float64       
 11  enterprise_to_revenue  4937 non-null   float64       
 12  ebitda_margins         5442 non-null   float64       
 13  mar

Clean up `data2` and create `years_to_ipo` feature

In [13]:
data2['ipo_year'] = data2['ipo_year'].fillna(2000).astype(int)


In [14]:
data2['founding'] = data2['founding'].fillna(2000).astype(int)


In [15]:
data2['years_to_ipo'] = data2['ipo_year'] - data2['founding']


**Modeling 2 Dataset**

In [16]:
data2 = data2[['ticker',
               'trade_date', 'ipo_year',
                      'day_1_%',
                      'month_7_%',
                      'sector',
                      'vc_dummy',
                      'rollup',
                      'dual',
                      'internet',
                      'years_to_ipo',
                      'enterprise_to_revenue',
                      'ebitda_margins',
                      'profit_margins',
                      'gross_margins',
                      'revenue_growth',
                      'total_revenue',
                      'operating_cashflow'
                    ]]
data2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6435 entries, 0 to 6434
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ticker                 6435 non-null   object        
 1   trade_date             6434 non-null   datetime64[ns]
 2   ipo_year               6435 non-null   int64         
 3   day_1_%                6435 non-null   float64       
 4   month_7_%              2392 non-null   float64       
 5   sector                 5451 non-null   object        
 6   vc_dummy               4820 non-null   object        
 7   rollup                 4820 non-null   object        
 8   dual                   4820 non-null   object        
 9   internet               4820 non-null   object        
 10  years_to_ipo           6435 non-null   int64         
 11  enterprise_to_revenue  4937 non-null   float64       
 12  ebitda_margins         5442 non-null   float64       
 13  pro

In [17]:
data2= data2.groupby(['ticker']).max().reset_index()

In [18]:
data2 = data2[data2['vc_dummy'].notna()]


In [19]:
data2 = data2[data2['day_1_%'].replace([np.inf, -np.inf], np.nan).notna()]
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4716 entries, 1 to 6433
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ticker                 4716 non-null   object        
 1   trade_date             4715 non-null   datetime64[ns]
 2   ipo_year               4716 non-null   int64         
 3   day_1_%                4716 non-null   float64       
 4   month_7_%              1563 non-null   float64       
 5   sector                 4340 non-null   object        
 6   vc_dummy               4716 non-null   object        
 7   rollup                 4716 non-null   object        
 8   dual                   4716 non-null   object        
 9   internet               4716 non-null   object        
 10  years_to_ipo           4716 non-null   int64         
 11  enterprise_to_revenue  3865 non-null   float64       
 12  ebitda_margins         4329 non-null   float64       
 13  pro

**Feature Engineer Dependent Variable `Successful`**

In [20]:
data2['gross_margins'].describe().T

count    4329.000000
mean        0.197415
std         0.414192
min        -2.943020
25%         0.000000
50%         0.000000
75%         0.410980
max         8.502710
Name: gross_margins, dtype: float64

In [21]:
data2['enterprise_to_revenue'].describe().T

count      3865.000000
mean        300.886847
std       10128.686824
min       -4069.345000
25%           0.000000
50%           0.542000
75%           4.270000
max      560179.440000
Name: enterprise_to_revenue, dtype: float64

In [22]:
def profitable(value):
    if value >=.65:
        return 1
    else:
        return 0
data2['profitable'] = data2['gross_margins'].map(lambda x: profitable(x))


In [23]:
data2['profitable'].value_counts()  

0    4145
1     571
Name: profitable, dtype: int64

In [24]:
def valuable(value):
    if value >=4.27:
        return 1
    else:
        return 0
data2['valuable'] = data2['enterprise_to_revenue'].map(lambda x: valuable(x))


In [25]:
data2['valuable'].value_counts()

0    3749
1     967
Name: valuable, dtype: int64

In [26]:
data2['day_1_%'] = data2['day_1_%'].replace([np.inf, -np.inf],0.0)

In [27]:
data2['month_7_%'] = data2['month_7_%'].replace([np.inf, -np.inf],0.0)

In [28]:
data2['success']  = data2['valuable']+data2['profitable']
data2['success'].value_counts()

0    3565
1     764
2     387
Name: success, dtype: int64

In [29]:
def success(value):
    if value > 0:
        return 1
        
    if value ==0:
        return 0

    
data2['successful'] = data2['success'].map(lambda x: success(x))

In [30]:
data2['successful'].value_counts()

0    3565
1    1151
Name: successful, dtype: int64

In [31]:
data2 = data2[data2['successful'].notna()]


In [32]:
data2 = data2[data2['sector'].notna()]


In [33]:
data2 = data2[data2['enterprise_to_revenue'].notna()]


In [34]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3848 entries, 1 to 6433
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ticker                 3848 non-null   object        
 1   trade_date             3847 non-null   datetime64[ns]
 2   ipo_year               3848 non-null   int64         
 3   day_1_%                3848 non-null   float64       
 4   month_7_%              1323 non-null   float64       
 5   sector                 3848 non-null   object        
 6   vc_dummy               3848 non-null   object        
 7   rollup                 3848 non-null   object        
 8   dual                   3848 non-null   object        
 9   internet               3848 non-null   object        
 10  years_to_ipo           3848 non-null   int64         
 11  enterprise_to_revenue  3848 non-null   float64       
 12  ebitda_margins         3809 non-null   float64       
 13  pro

In [35]:
data2_viz = data2

In [36]:
data2 = data2[['ticker',
                      'sector',
                      'vc_dummy',
                      'rollup',
                      'dual',
                      'internet',
                      'years_to_ipo',
                      'day_1_%',
                      'month_7_%',
                      'successful']]
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3848 entries, 1 to 6433
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ticker        3848 non-null   object 
 1   sector        3848 non-null   object 
 2   vc_dummy      3848 non-null   object 
 3   rollup        3848 non-null   object 
 4   dual          3848 non-null   object 
 5   internet      3848 non-null   object 
 6   years_to_ipo  3848 non-null   int64  
 7   day_1_%       3848 non-null   float64
 8   month_7_%     1323 non-null   float64
 9   successful    3848 non-null   int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 330.7+ KB


In [37]:
joblib.dump(data2_viz,"data2_viz.pkl")

['data2_viz.pkl']

In [38]:
joblib.dump(data2,"data2.pkl")

['data2.pkl']